In [1]:
import numpy as np
import pandas as pd
import time

import requests                     #This is for scraping
from bs4 import BeautifulSoup       #This is for scraping
import webbrowser                   #Scraping web
import csv                          #This helps generating dataframes in pandas
import datetime                     #For operations regarding dates in the porject
from unidecode import unidecode     #This is for coding hungarian characters like "ö", "í" to o and i when needed
import unicodedata                  #same as previous

In [2]:
import selenium                     #this is for scraping a webpage
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import io 
from selenium.webdriver.common.keys import Keys 

In [3]:
#This is the database that I want to upgrade with the scraped content
df = pd.read_excel("data.xlsx",squeeze=True, encoding ="utf-8",dtype=str)
index = df.index
values = df.values
df.head()

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995


In [4]:

Committee_Codebook_2014_2018 = {
    197 : "Törvényalkotási bizottság",
    198 : "Gazdasági bizottság",
    199 : "Költségvetési bizottság", 
    200 : "Kulturális bizottság",
    201 : "Népjóléti bizottság",
    202 : "Igazságügyi bizottság",
    203 : "Külügyi bizottság",
    204 : "Mezőgazdasági bizottság ", 
    205 : "Vállalkozásfejlesztési bizottság",
    206 : "Fenntartható fejlődés bizottsága",
    207 : "Európai ügyek bizottsága",
    208 : "Honvédelmi és rendészeti bizottság",
    209 : "Nemzeti összetartozás bizottsága",
    210 : "Nemzetbiztonsági bizottság",
    211 : "Mentelmi bizottság",
    212 : "Az Alkotmánybíróság tagjait jelölő eseti bizottság",
    213 : "Magyarországi nemzetiségek bizottsága",
}

Committee_code = list(Committee_Codebook_2014_2018.keys())
Committee_list = list(Committee_Codebook_2014_2018.values())

In [5]:
#selenium login start
#It seems like we need selenium, since we got a dynamic webpage... unlucky
driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
html_source = driver.page_source #Makes soup
soup = BeautifulSoup(html_source, 'html.parser')
#print(soup.prettify())

In [6]:
#Let's see, what kind of links does the page contain. The goal is to click on the links of the Parliament members.
#First, let's make the list of all the urls. 
url_list = []
for url in soup.find_all('a'):
    url_list.append(url)

#Now, let's take a look on our data. 
for i in range (len(url_list)):
    print(i)
    print(url_list[i])


0
<a href="#main-content">Ugrás a tartalomhoz</a>
1
<a href="https://www.parlament.hu/web/guest/orszaggyules">
<span>ORSZÁGGYŰLÉS</span>
</a>
2
<a href="https://www.parlament.hu/web/guest/bevezetes-jogi-szabalyzas1" role="menuitem">
<span>Bevezetés, jogi szabályozás</span>
</a>
3
<a href="https://www.parlament.hu/web/guest/mukodes-feladatok1" role="menuitem">
<span>Működés, feladatok</span>
</a>
4
<a href="https://www.parlament.hu/web/guest/nemzetpolitika" role="menuitem">
<span>Nemzetpolitika</span>
</a>
5
<a href="https://www.parlament.hu/web/guest/europai-unios-tevekenyseg" role="menuitem">
<span>Európai uniós tevékenység</span>
</a>
6
<a href="https://www.parlament.hu/web/guest/interparlamentaris-unio1" role="menuitem">
<span>Az Interparlamentáris Unió</span>
</a>
7
<a href="https://www.parlament.hu/web/guest/az-orszaggyules-barati-tagozatai" role="menuitem">
<span>Az Országgyűlés baráti tagozatai</span>
</a>
8
<a href="https://www.parlament.hu/web/orszaghaz">
<span>Országház</span

In [7]:
#At Access Date: 19/09/2020, the parliamentary members from A to Z are Ander Balázs to Zsigó Róbert. 
#Ander Balázs has a link at the 231 number, Zsigó has at 464, so I will only keep these elements in the url_list. 
#It's important, that this might change in the future, but right now, my goal is to find and click on the links with the names. 
url_list = url_list[231:464]
print(url_list)

[<a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&amp;p_p_lifecycle=1&amp;p_p_state=normal&amp;p_p_mode=view&amp;p_auth=yYIhXrcA&amp;_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D">Ander Balázs</a>, <a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&amp;p_p_lifecycle=1&amp;p_p_state=normal&amp;p_p_mode=view&amp;p_auth=yYIhXrcA&amp;_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da018%26p_ckl%3D40%26p_osszefuz%3D">Dr. Apáti István</a>, <a class="" href="https://www.parlament.hu:443/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9

In [8]:
#I convert the url to string
for i in range (len(url_list)):
    url_list[i] = str(url_list[i])

In [9]:
#The href links won't direct me to the parliament member's page with request, I have to click on it. 
#All of the href elements has a name at the very end, thats what you see when you open the page. 
#I will collect all the names of the parliament members from the page. 
Names = []
for i in range(len(url_list)):
    if len(url_list[i]) > 400:
        name = (url_list[i][405:(len(url_list[i])-4)])
        Names.append(name)
#So, what do we have? 
print(Names)

['Ander Balázs', 'Dr. Apáti István', 'Dr. Aradszki András', 'Ágh Péter', 'B. Nagy László', 'Bajnai Gordon', 'Balczó Zoltán', 'Balla György', 'Balla Mihály', 'Balog Zoltán', 'Bana Tibor', 'Bangóné Borbély Ildikó', 'Bánki Erik', 'Bányai Gábor', 'Dr. Bárándy Gergely', 'Bartos Mónika', 'Becsó Zsolt', 'Bencsik János', 'Dr. Bene Ildikó', 'Bíró Márk', 'Bodó Sándor', 'Boldog István', 'Bóna Zoltán', 'Dr. Botka László', 'Budai Gyula', 'Burány Sándor', 'Csenger-Zalán Zsolt', 'Cseresnyés Péter', 'Csizi Péter', 'Csöbör Katalin', 'Czerván György', 'Dr. Czomba Sándor', 'Czunyiné Dr. Bertalan Judit', 'Dankó Béla', 'Demeter Márta', 'Demeter Zoltán', 'Dunai Mónika', 'Dúró Dóra', 'Egyed Zsolt', 'Farkas Flórián', 'Farkas Gergely', 'Farkas Sándor', 'Dr. Fazekas Sándor', 'Firtl Mátyás', 'Fodor Gábor', 'Dr. Fónagy János', 'Font Sándor', 'Földi László', 'Gaal Gergely', 'Dr. Galambos Dénes', 'Gelencsér Attila', 'Gőgös Zoltán', 'Gulyás Gergely', 'Gúr Nándor', 'Gyopáros Alpár', 'Gyöngyösi Márton', 'Győrffy Baláz

#### In the next cell, I will make all the functions that I am tasked with to scrape down, when opening the page of each politician. F.e.: I get their party membership, I get if they were elected on a national list, or in an electorate district, etc. 

In [10]:
#Gives the party membership position of the person between 2014 and 2018
def simplify_text(text):
    #Changes the text to sufficient formats for the dataframe    
    text = text.replace("Dr. ", "") #removes Dr. tags
    text = text.replace("megye ","")
    text = text.replace(" ", "_")   #changes spaces to underlines
    text = text.lower()             #removes capital letters
    normalized=unicodedata.normalize('NFD', text) #this and next line: Hungarian special charcters are decoded to English
    text = u"".join([c for c in normalized if not unicodedata.combining(c)])
    return text

def get_all_table_data_page(soup):
    all_table_data_page = []
    for element in soup.find_all('td'):
        element = str(element)
        all_table_data_page.append(element)
    
    return all_table_data_page

def get_party(soup):
    trigger = 0
    for element in soup.find_all('td'):
        element = str(element)
        if trigger == 1:
            result = element[4:-5]
            trigger = 2
        if element == "<td>2014-2018</td>" and trigger == 0:
            trigger = 1
    result = simplify_text(result)
    return result

#Gives if someone supports government or not. 
#I am decoding Party membership to a dummy variable, where 1 relates to government parties, 0 relates to opposition parties
def get_gov_support(party):
    result = 0
    if party == "fidesz" or party == "kdnp":
        result = 1
    return result

#Gives the exact seat of the politician.
def get_exact_seat(soup):
    result = 99999
    all_table_data_page = get_all_table_data_page(soup)
    
    for i in range (len(all_table_data_page)):
        if all_table_data_page[i] == '<td>2014.04.06.</td>':
            result = all_table_data_page[i+1][4:-9] 
    
    #Don't if this is necessary
    if type(result) == str:        
        result = simplify_text(result)
    return(result)  
    
#Gives a raw data, in which committee a parliamentary politician has been involved in.
def get_committee_data(Committee_list, soup, toprint = False):
    all_page_data = get_all_table_data_page(soup)
        
#Part 2: gives a data in the seeked format for us, and return 99995, if the politician has not been in committee    
    result = []
    for committee in Committee_list:
        for i in range (len(all_page_data)):

            if committee in all_page_data[i]:
                result.append(committee)
                result.append(all_page_data[i+1][4:-5])
                result.append(all_page_data[i+2][4:-5])
                result.append(all_page_data[i+3][4:-5])
    if result == []:
        result = 99995
    return result

def get_seniority(soup):
    all_page_data = get_all_table_data_page(soup)
    
    cycles = ['1990-94',
              '1994-98',
              '1998-2002',
              '2002-2006',
              '2006-2010',
              '2010-2014',
              '2014-2018']

    seniority = 0
    checker = -1
    for i in range(len(cycles)):
        for element in all_page_data:
            if cycles[i] in element:
                checker = i
        if checker == i:
            seniority += 1
    return seniority

#This one gets the data in rows, sufficient for loading it into the pandas dataframe. 
def give_pandas_committee_data (name, committee_data, source_url, party, gov_support, exact_seat, seat_type, seniority):
    result = []
#Part 1: Changes the names to sufficient format for the dataframe    
    name = simplify_text(name)
    
#Part 2: if the parliamentary member has not been in any committee, his committee_data will be 99995
#How 99995? See give_pandas_committee_data... It will generate the new df row
    if committee_data == 99995:
        print(name)
        new_row = {'name': name, 
                   'gov_support': gov_support, 
                   'source': source_url,
                   'committee_membership': 99995,
                   'cm_from_year': 99995,
                   'cm_from_month': 99995,
                   'cm_from_day': 99995,
                   'cm_to_year': 99995,
                   'cm_to_month': 99995,
                   'cm_to_day': 99995,
                   'seat_type': seat_type,
                   'exact_seat' : exact_seat,
                   'electoral_cycle': '2014-2018',
                   'seniority' : seniority,
                  }
        print(new_row)
        result.append(new_row)

#Part 3: If the data is not 99995 (aka. empty) then all the data schould consist: 
#which committee [i*4]
#role [i*4+1]
#start of membership [i*4+2]
#end of membership [i*4+3]
#NOTE!!! This is only true for the results we are looking for, some "sub-committee" might be included in the data
#Subcommittees are not part of this research, so we will eliminate them later
    else:
        for i in range (len(committee_data)//4):
            committee_data[4*i] = Committee_code[Committee_list.index(committee_data[4*i])]
            new_row = {'name': name, 
                       'gov_support': gov_support, 
                       'source': source_url,
                       'committee_membership': committee_data[4*i],
                       'cm_from_year': committee_data[4*i+2][0:4],
                       'cm_from_month': committee_data[4*i+2][5:7],
                       'cm_from_day': committee_data[4*i+2][8:10],
                       'cm_to_year': committee_data[4*i+3][0:4],
                       'cm_to_month': committee_data[4*i+3][5:7],
                       'cm_to_day': committee_data[4*i+3][8:10],
                       'seat_type': seat_type,
                       'exact_seat' : exact_seat,
                       'electoral_cycle' : '2014-2018',
                       'seniority' : seniority,
                      }
            print(new_row)
            result.append(new_row)
    return result

In [11]:
#It is time for the scrape loop, I click on the links with the Parliamentary members. 

all_rows = [] #I want all the rows in data structure: [Politician 1[[Comm. 1],[Comm. 2]],Politican 2[Comm. 1],[Comm. 2]]
#In the target dataframe: 1 committee membership is one line, not one politician, therefore we need to spare the data more

#This loop open up every politicians page
for i in range(len(Names)):
    
    driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
    #Next line: clicks on the element on the page, that contains the matching name. 
    link = driver.find_element_by_link_text(Names[i])
    link.click()
    source_url = driver.current_url
    html_source = driver.page_source #Makes soup
    soup = BeautifulSoup(html_source, 'html.parser')
    
    party = get_party(soup)
    seniority = get_seniority(soup)
    exact_seat = get_exact_seat(soup)
    gov_support = get_gov_support(party)
    seat_type = 'smd'
    if exact_seat == 'orszagos_l':
        seat_type = 'pl'
    if exact_seat == 'orszagos_l':
        exact_seat = party
        
    committee_data = get_committee_data(Committee_list, soup)
    new_rows = give_pandas_committee_data(Names[i], committee_data, source_url, party, gov_support,exact_seat,seat_type,seniority)
    all_rows.append(new_rows)
    
    #Sleep a bit, dude! 
    time.sleep(0.5)   

{'name': 'ander_balazs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 200, 'cm_from_year': '2018', 'cm_from_month': '06', 'cm_from_day': '11', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'pl', 'exact_seat': 'jobbik', 'electoral_cycle': '2014-2018', 'seniority': 1}
{'name': 'ander_balazs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fint

{'name': 'agh_peter', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da011%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 197, 'cm_from_year': '2020', 'cm_from_month': '05', 'cm_from_day': '05', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'smd', 'exact_seat': 'vas_2', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'agh_peter', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%

{'name': 'balla_gyorgy', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db007%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 197, 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '2018', 'cm_to_month': '10', 'cm_to_day': '15', 'seat_type': 'pl', 'exact_seat': 'fidesz', 'electoral_cycle': '2014-2018', 'seniority': 5}
{'name': 'balla_gyorgy', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2

balog_zoltan
{'name': 'balog_zoltan', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db052%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 99995, 'cm_from_year': 99995, 'cm_from_month': 99995, 'cm_from_day': 99995, 'cm_to_year': 99995, 'cm_to_month': 99995, 'cm_to_day': 99995, 'seat_type': 'pl', 'exact_seat': 'fidesz', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'bana_tibor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z

{'name': 'banyai_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db053%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 205, 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'smd', 'exact_seat': 'bacs-kiskun_5', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'banyai_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairActio

...

{'name': 'varga_mihaly', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv331%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 198, 'cm_from_year': '1998', 'cm_from_month': '06', 'cm_from_day': '25', 'cm_to_year': '1998', 'cm_to_month': '07', 'cm_to_day': '07', 'seat_type': 'smd', 'exact_seat': 'budapest_4', 'electoral_cycle': '2014-2018', 'seniority': 7}
{'name': 'vargha_tamas', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairActi

{'name': 'vecsey_laszlo', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv044%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 197, 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'smd', 'exact_seat': 'pest_6', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'vecsey_laszlo', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2F

{'name': 'vigh_laszlo', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv038%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 202, 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'smd', 'exact_seat': 'zala_1', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'vigh_laszlo', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finte

{'name': 'vona_gabor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv058%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 203, 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'pl', 'exact_seat': 'jobbik', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'vona_gabor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fint

{'name': 'zombor_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dz008%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 201, 'cm_from_year': '2018', 'cm_from_month': '05', 'cm_from_day': '08', 'cm_to_year': '\xa0', 'cm_to_month': '', 'cm_to_day': '', 'seat_type': 'smd', 'exact_seat': 'bacs-kiskun_2', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'zombor_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairActio

In [12]:
import datetime

In [13]:
election_2014 = datetime.datetime(2014, 4, 6)
election_2018 = datetime.datetime(2018, 4, 8)
            
for rows in all_rows:
    for row in rows:
        values = list(row.values())
        if values [3] == 99995:
            df = df.append(row, ignore_index=True)
        else:
            start = (values[4]+'-'+values[5]+'-'+values[6])
            end   = (values[7]+'-'+values[8]+'-'+values[9])

            #Remove duplicate data
            if '20' in values[4] and '20' in values [7]:
                start = datetime.datetime.strptime(start, '%Y-%m-%d')
                end = datetime.datetime.strptime(end, '%Y-%m-%d')

                if election_2014 <= start <= election_2018:
                    print(row)
                    df = df.append(row, ignore_index=True)

{'name': 'ander_balazs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 201, 'cm_from_year': '2016', 'cm_from_month': '09', 'cm_from_day': '12', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'pl', 'exact_seat': 'jobbik', 'electoral_cycle': '2014-2018', 'seniority': 1}
{'name': 'ander_balazs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2

{'name': 'banki_erik', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db009%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 198, 'cm_from_year': '2015', 'cm_from_month': '10', 'cm_from_day': '19', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'pl', 'exact_seat': 'fidesz', 'electoral_cycle': '2014-2018', 'seniority': 5}
{'name': 'banyai_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fi

{'name': 'bencsik_janos', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db034%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 206, 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2014', 'cm_to_month': '06', 'cm_to_day': '02', 'seat_type': 'smd', 'exact_seat': 'komarom-esztergom_1', 'electoral_cycle': '2014-2018', 'seniority': 4}
{'name': 'bencsik_janos', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4

{'name': 'budai_gyula', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db076%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 202, 'cm_from_year': '2014', 'cm_from_month': '06', 'cm_from_day': '07', 'cm_to_year': '2014', 'cm_to_month': '06', 'cm_to_day': '23', 'seat_type': 'pl', 'exact_seat': 'fidesz', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'budai_gyula', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fi

{'name': 'czunyine_bertalan_judit', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dc029%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 202, 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2014', 'cm_to_month': '06', 'cm_to_day': '02', 'seat_type': 'smd', 'exact_seat': 'komarom-esztergom_3', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'czunyine_bertalan_judit', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy

...

{'name': 'tapolczai_gergely', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt044%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 197, 'cm_from_year': '2015', 'cm_from_month': '02', 'cm_from_day': '16', 'cm_to_year': '2018', 'cm_to_month': '05', 'cm_to_day': '07', 'seat_type': 'pl', 'exact_seat': 'fidesz', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'tapolczai_gergely', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pai

{'name': 'tukacs_istvan', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt028%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 197, 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2016', 'cm_to_month': '11', 'cm_to_day': '07', 'seat_type': 'pl', 'exact_seat': 'mszp', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'tukacs_istvan', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2

{'name': 'varju_laszlo', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv018%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 198, 'cm_from_year': '2014', 'cm_from_month': '07', 'cm_from_day': '04', 'cm_to_year': '2017', 'cm_to_month': '10', 'cm_to_day': '02', 'seat_type': 'smd', 'exact_seat': 99999, 'electoral_cycle': '2014-2018', 'seniority': 4}
{'name': 'varju_laszlo', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fi

{'name': 'velez_arpad', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv041%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 199, 'cm_from_year': '2016', 'cm_from_month': '09', 'cm_from_day': '19', 'cm_to_year': '2016', 'cm_to_month': '11', 'cm_to_day': '21', 'seat_type': 'pl', 'exact_seat': 'mszp', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'velez_arpad', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Fint

{'name': 'zombor_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dz008%26p_ckl%3D40%26p_osszefuz%3D', 'committee_membership': 201, 'cm_from_year': '2014', 'cm_from_month': '05', 'cm_from_day': '06', 'cm_to_year': '2014', 'cm_to_month': '06', 'cm_to_day': '23', 'seat_type': 'smd', 'exact_seat': 'bacs-kiskun_2', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'zombor_gabor', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=yYIhXrcA&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairA

In [14]:
df

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995
3,NaN,2014-2018,https://www.parlament.hu/web/guest/kepviselok-...,ander_balazs,NaN,0,jobbik,201,NaN,1,...,2016,07,05,2018,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2014-2018,https://www.parlament.hu/web/guest/kepviselok-...,ander_balazs,NaN,0,jobbik,209,NaN,1,...,2014,12,09,2016,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,NaN,2014-2018,https://www.parlament.hu/web/guest/kepviselok-...,z._karpat_daniel,NaN,0,jobbik,205,NaN,2,...,2016,07,05,2018,NaN,NaN,NaN,NaN,NaN,NaN
363,NaN,2014-2018,https://www.parlament.hu/web/guest/kepviselok-...,zombor_gabor,NaN,1,bacs-kiskun_2,201,NaN,3,...,2014,23,06,2014,NaN,NaN,NaN,NaN,NaN,NaN
364,NaN,2014-2018,https://www.parlament.hu/web/guest/kepviselok-...,zombor_gabor,NaN,1,bacs-kiskun_2,201,NaN,3,...,2015,07,05,2018,NaN,NaN,NaN,NaN,NaN,NaN
365,NaN,2014-2018,https://www.parlament.hu/web/guest/kepviselok-...,zsigo_robert,NaN,1,bacs-kiskun_6,203,NaN,5,...,2014,07,06,2014,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
 df.drop_duplicates(keep = False, inplace = True) 

In [16]:
import openpyxl

In [17]:
df.to_excel("final_scrape_coded_committee_data.xlsx") 

In [18]:
#This is the database that I want to upgrade with the scraped content
df = pd.read_excel("data.xlsx",squeeze=True, encoding ="utf-8",dtype=str)
index = df.index
values = df.values
df.head()

,id,electoral_cycle,source,name,sex,gov_support,exact_seat,committee_membership,party_hierarchy,seniority,...,cm_from_year,cm_to_day,cm_to_month,cm_to_year,ph_from_day,ph_from_month,ph_from_year,ph_to_day,ph_to_month,ph_to_year
0,a182_1990_003_19921020_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,abraham_tibor,m,0,pest,3,99995,1,...,1992,27,6,1994,99995,99995,99995,99995,99995,99995
1,a302_1990_001_19900503_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,1,99995,1,...,1990,24,3,1992,99995,99995,99995,99995,99995,99995
2,a302_1990_009_19920324_0,1990-1994,http://www.parlament.hu/kepviselok-elozo-ciklu...,ader_janos,m,0,fidesz,9,99995,1,...,1992,8,4,1992,99995,99995,99995,99995,99995,99995


In [19]:
#selenium login start
#It seems like we need selenium, since we got a dynamic webpage... unlucky
driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
html_source = driver.page_source #Makes soup
soup = BeautifulSoup(html_source, 'html.parser')
#print(soup.prettify())

In [20]:
def get_party_hierarchy(soup):
    all_page_data = get_all_table_data_page(soup)
    result = []
    for i in range(len(all_page_data)):
        if 'frakcióvez. h.' in all_page_data[i]:
            if '20' in all_page_data[i+1] and '20' in all_page_data[i+2]:
                result.append(all_page_data[i][4:-5])
                result.append(all_page_data[i+1][4:-5])
                result.append(all_page_data[i+2][4:-5])
            
        if 'frakcióvezető' in all_page_data[i]:
            result.append(all_page_data[i][4:-5])
            result.append(all_page_data[i+1][4:-5])
            result.append(all_page_data[i+2][4:-5])
    if result == []:
        result = 99995
    return result

def give_pandas_party_hierarchy_data (name, party_hierarchy, source_url, party, gov_support, exact_seat, seat_type, seniority):
    result = []
#Part 1: Changes the names to sufficient format for the dataframe    
    name = simplify_text(name)
    
#Part 2: if the parliamentary member has not been in any committee, his committee_data will be 99995
#How 99995? See give_pandas_committee_data... It will generate the new df row
    if party_hierarchy == 99995:
        new_row = {'name': name, 
                   'gov_support': gov_support, 
                   'source': source_url,
                   'party_hierarchy': 99995,
                   'ph_from_year': 99995,
                   'ph_from_month': 99995,
                   'ph_from_day': 99995,
                   'ph_to_year': 99995,
                   'ph_to_month': 99995,
                   'ph_to_day': 99995,
                   'seat_type': seat_type,
                   'exact_seat' : exact_seat,
                   'electoral_cycle': '2014-2018',
                   'seniority' : seniority,
                  }
        print(new_row)
        result.append(new_row)

    else:
        for i in range (len(party_hierarchy)//3):
            new_row = {'name': name, 
                       'gov_support': gov_support, 
                       'source': source_url,
                       'party_hierarchy': party_hierarchy[3*i],
                       'ph_from_year': party_hierarchy[3*i+1][0:4],
                       'ph_from_month': party_hierarchy[3*i+1][5:7],
                       'ph_from_day': party_hierarchy[3*i+1][8:10],
                       'ph_to_year': party_hierarchy[3*i+2][0:4],
                       'ph_to_month': party_hierarchy[3*i+2][5:7],
                       'ph_to_day': party_hierarchy[3*i+2][8:10],
                       'seat_type': seat_type,
                       'exact_seat' : exact_seat,
                       'electoral_cycle' : '2014-2018',
                       'seniority' : seniority,
                      }
            print(new_row)
            result.append(new_row)
            
    return result

In [21]:
#It is time for the scrape loop, I click on the links with the Parliamentary members. 

all_rows = [] #I want all the rows in data structure: [Politician 1[[Comm. 1],[Comm. 2]],Politican 2[Comm. 1],[Comm. 2]]
#In the target dataframe: 1 committee membership is one line, not one politician, therefore we need to spare the data more

#This loop open up every politicians page
for i in range(len(Names)):
    
    driver.get('https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=MbCEKk7P&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_lis%3Fp_ckl%3D40')
    #Next line: clicks on the element on the page, that contains the matching name. 
    link = driver.find_element_by_link_text(Names[i])
    link.click()
    source_url = driver.current_url
    html_source = driver.page_source #Makes soup
    soup = BeautifulSoup(html_source, 'html.parser')
    
    party = get_party(soup)
    seniority = get_seniority(soup)
    exact_seat = get_exact_seat(soup)
    gov_support = get_gov_support(party)
    seat_type = 'smd'
    if exact_seat == 'orszagos_l':
        seat_type = 'pl'
    if exact_seat == 'orszagos_l':
        exact_seat = party
    
    party_hierarchy = get_party_hierarchy(soup)
    
    new_rows = give_pandas_party_hierarchy_data(Names[i], party_hierarchy, source_url, party, gov_support,exact_seat,seat_type, seniority)
    all_rows.append(new_rows)
    
    #Sleep a bit, dude! 
    time.sleep(0.5)   

{'name': 'ander_balazs', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Da716%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 99995, 'ph_from_year': 99995, 'ph_from_month': 99995, 'ph_from_day': 99995, 'ph_to_year': 99995, 'ph_to_month': 99995, 'ph_to_day': 99995, 'seat_type': 'pl', 'exact_seat': 'jobbik', 'electoral_cycle': '2014-2018', 'seniority': 1}
{'name': 'apati_istvan', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2F

{'name': 'balczo_zoltan', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db022%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 'frakcióvez. h.', 'ph_from_year': '2010', 'ph_from_month': '05', 'ph_from_day': '14', 'ph_to_year': '2014', 'ph_to_month': '05', 'ph_to_day': '05', 'seat_type': 'pl', 'exact_seat': 'jobbik', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'balla_gyorgy', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pair

{'name': 'bana_tibor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db084%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 99995, 'ph_from_year': 99995, 'ph_from_month': 99995, 'ph_from_day': 99995, 'ph_to_year': 99995, 'ph_to_month': 99995, 'ph_to_day': 99995, 'seat_type': 'pl', 'exact_seat': 'jobbik', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'bangone_borbely_ildiko', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAc

{'name': 'bene_ildiko', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dj012%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 99995, 'ph_from_year': 99995, 'ph_from_month': 99995, 'ph_from_day': 99995, 'ph_to_year': 99995, 'ph_to_month': 99995, 'ph_to_day': 99995, 'seat_type': 'smd', 'exact_seat': 'jasz-nagykun-szolnok_1', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'biro_mark', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_p

{'name': 'budai_gyula', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Db076%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 99995, 'ph_from_year': 99995, 'ph_from_month': 99995, 'ph_from_day': 99995, 'ph_to_year': 99995, 'ph_to_month': 99995, 'ph_to_day': 99995, 'seat_type': 'pl', 'exact_seat': 'fidesz', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'burany_sandor', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2F

...

{'name': 'toth_csaba', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dt045%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 'frakcióvez. h.', 'ph_from_year': '2016', 'ph_from_month': '07', 'ph_from_day': '06', 'ph_to_year': '2018', 'ph_to_month': '05', 'ph_to_day': '07', 'seat_type': 'smd', 'exact_seat': 'budapest_8', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'toth_csaba', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pair

{'name': 'vadai_agnes', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv013%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 'frakcióvez. h.', 'ph_from_year': '2007', 'ph_from_month': '05', 'ph_from_day': '21', 'ph_to_year': '2007', 'ph_to_month': '05', 'ph_to_day': '23', 'seat_type': 'pl', 'exact_seat': 'fuggetlen', 'electoral_cycle': '2014-2018', 'seniority': 4}
{'name': 'vadai_agnes', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pair

{'name': 'varga_laszlo', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv032%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 'frakcióvez. h.', 'ph_from_year': '2016', 'ph_from_month': '07', 'ph_from_day': '06', 'ph_to_year': '2018', 'ph_to_month': '05', 'ph_to_day': '07', 'seat_type': 'smd', 'exact_seat': 'borsod-abauj-zemplen_2', 'electoral_cycle': '2014-2018', 'seniority': 3}
{'name': 'varga_laszlo', 'gov_support': 0, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9

{'name': 'vejkey_imre', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv055%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 'frakcióvez. h.', 'ph_from_year': '2014', 'ph_from_month': '05', 'ph_from_day': '06', 'ph_to_year': '2018', 'ph_to_month': '05', 'ph_to_day': '07', 'seat_type': 'pl', 'exact_seat': 'kdnp', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'vejkey_imre', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairActio

{'name': 'volner_pal', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8_pairAction=%2Finternet%2Fcplsql%2Fogy_kpv.kepv_adat%3Fp_azon%3Dv046%26p_ckl%3D40%26p_osszefuz%3D', 'party_hierarchy': 99995, 'ph_from_year': 99995, 'ph_from_month': 99995, 'ph_from_day': 99995, 'ph_to_year': 99995, 'ph_to_month': 99995, 'ph_to_day': 99995, 'seat_type': 'smd', 'exact_seat': 'komarom-esztergom_2', 'electoral_cycle': '2014-2018', 'seniority': 2}
{'name': 'witzmann_mihaly', 'gov_support': 1, 'source': 'https://www.parlament.hu/web/guest/kepviselok-elozo-ciklusbeli-adatai?p_p_id=hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8&p_p_lifecycle=1&p_p_state=normal&p_p_mode=view&p_auth=cGSMAkQO&_hu_parlament_cms_pair_portlet_PairProxy_INSTANCE_9xd2Wc9jP4z8

In [22]:
election_2014 = datetime.datetime(2014, 4, 6)
election_2018 = datetime.datetime(2018, 4, 8)
            
for rows in all_rows:
    checker = 0 #We need to check, if someone has been in party hierarchy earlier, but not in the 2014-2018 cycle
    for row in rows:
        values = list(row.values())
        print(values[0], values[3])
        if values [3] == 99995:
            df = df.append(row, ignore_index=True)
            checker = 1
        else:
            start = (values[4]+'-'+values[5]+'-'+values[6])
            end   = (values[7]+'-'+values[8]+'-'+values[9])

            #Remove duplicate data
            if '20' in values[4] and '20' in values [7]:
                start = datetime.datetime.strptime(start, '%Y-%m-%d')
                end = datetime.datetime.strptime(end, '%Y-%m-%d')

                if election_2014 <= start <= election_2018:
                    df = df.append(row, ignore_index=True)
                    checker = 1
    if checker == 0:
        row.update({'party_hierarchy': 99995,
                    'ph_to_year':99995,
                    'ph_to_month':99995,
                    'ph_to_day':99995,
                    'ph_from_year':99995,
                    'ph_from_month':99995,
                    'ph_from_day':99995,})
        df = df.append(row, ignore_index=True)

ander_balazs 99995
apati_istvan 99995
aradszki_andras frakcióvez. h.
aradszki_andras frakcióvez. h.
aradszki_andras frakcióvez. h.
agh_peter frakcióvez. h.
agh_peter frakcióvez. h.
agh_peter frakcióvez. h.
agh_peter frakcióvez. h.
agh_peter frakcióvez. h.
agh_peter frakcióvez. h.
b._nagy_laszlo 99995
bajnai_gordon 99995
balczo_zoltan frakcióvez. h.
balla_gyorgy frakcióvez. h.
balla_gyorgy frakcióvez. h.
balla_gyorgy frakcióvez. h.
balla_gyorgy frakcióvez. h.
balla_gyorgy frakcióvez. h.
balla_gyorgy frakcióvez. h.
balla_mihaly frakcióvez. h.
balla_mihaly frakcióvez. h.
balla_mihaly frakcióvez. h.
balla_mihaly frakcióvez. h.
balla_mihaly frakcióvez. h.
balog_zoltan frakcióvez. h.
balog_zoltan frakcióvez. h.
balog_zoltan frakcióvez. h.
balog_zoltan frakcióvez. h.
bana_tibor 99995
bangone_borbely_ildiko frakcióvez. h.
banki_erik frakcióvez. h.
banki_erik frakcióvez. h.
banki_erik frakcióvez. h.
banki_erik frakcióvez. h.
banki_erik frakcióvez. h.
banyai_gabor 99995
barandy_gergely frakcióve

rig_lajos 99995
riz_gabor 99995
rogan_antal frakcióvezető
rogan_antal frakcióvezető
rogan_antal frakcióvez. h.
rogan_antal frakcióvez. h.
rogan_antal frakcióvez. h.
rubovszky_gyorgy frakcióvez. h.
rubovszky_gyorgy frakcióvez. h.
rubovszky_gyorgy frakcióvez. h.
salacz_laszlo 99995
sallai_r._benedek 99995
schiffer_andras frakcióvezető
schiffer_andras frakcióvezető
schiffer_andras frakcióvezető
schiffer_andras frakcióvezető
schmuck_erzsebet frakcióvez. h.
schmuck_erzsebet frakcióvez. h.
schmuck_erzsebet frakcióvezető
schmuck_erzsebet frakcióvez. h.
schmuck_erzsebet frakcióvez. h.
selmeczi_gabriella frakcióvez. h.
selmeczi_gabriella frakcióvez. h.
selmeczi_gabriella frakcióvez. h.
semjen_zsolt frakcióvez. h.
semjen_zsolt frakcióvezető
sesztak_miklos 99995
simicsko_istvan frakcióvezető
simicsko_istvan frakcióvez. h.
simicsko_istvan frakcióvez. h.
simon_miklos 99995
simon_robert_balazs 99995
simonka_gyorgy frakcióvez. h.
sneider_tamas frakcióvez. h.
soltesz_miklos frakcióvez. h.
staudt_gabor

In [23]:
df.to_excel("final_scrape_hierarchy_data.xlsx") 

#### Future updates 1: merge committee list

In [ ]:
election_2014 = datetime.datetime(2014, 4, 6)
election_2018 = datetime.datetime(2018, 4, 8)
            
for rows in all_rows:
    toskiprow = False
    for i in range (len(rows)):
        if toskiprow == True:
            toskiprow = False
        else:
            if i != (len(rows)-1):

                values = list(rows[i].values())
                next_values = list(rows[i+1].values())
                if values [3] == 99995:
                    df = df.append(rows[i], ignore_index=True)
                    
                else:
                    start = (values[4]+'-'+values[5]+'-'+values[6])
                    end   = (values[7]+'-'+values[8]+'-'+values[9])
                    next_start = (next_values[4]+'-'+next_values[5]+'-'+next_values[6])
                    next_end = (next_values[7]+'-'+next_values[8]+'-'+next_values[9])
                    
                    print("Start:",start)
                    print("End:",end)
                    print("Next_start:",next_start)
                    print("Next_end:",next_end)
                    
                    if end == next_start and values[3] == next_values[3]:
                        toskiprow = True
                        
                        if '20' in values[4] and '20' in values [7]:
                            start = datetime.datetime.strptime(start, '%Y-%m-%d')
                            end = datetime.datetime.strptime(next_end, '%Y-%m-%d')
                            
                            if election_2014 <= start <= election_2018:
                                df = df.append(row, ignore_index=True)

                    else:
                        #Remove duplicate data
                        if '20' in values[4] and '20' in values [7]:
                            start = datetime.datetime.strptime(start, '%Y-%m-%d')
                            end = datetime.datetime.strptime(next_end, '%Y-%m-%d')

                            if election_2014 <= start <= election_2018:
                                df = df.append(row, ignore_index=True)
                            
            else:   
                values = list(rows[i].values())
                if values [3] == 99995:
                    df = df.append(rows[i], ignore_index=True)
                else:
                    start = (values[4]+'-'+values[5]+'-'+values[6])
                    end   = (values[7]+'-'+values[8]+'-'+values[9])
                    #Remove duplicate data
                    if '20' in values[4] and '20' in values [7]:
                        start = datetime.datetime.strptime(start, '%Y-%m-%d')
                        end = datetime.datetime.strptime(end, '%Y-%m-%d')

                        if election_2014 <= start <= election_2018:
                            df = df.append(row, ignore_index=True)

#### Future updates 2: merge hieararchy into committee data

In [ ]:
def get_party_hierarchy(soup,start,end):
    hierarchy = [99995,99995,99995,99995,99995,99995,99995,99995]
    for element in soup.find_all('th'):
        element = str(element) 
        if "Képviselőcsoport tisztség" in element:
            hierarchy[0] = 1
        
    if hierarchy[0] != 99995:
        all_page_data = []
        for element in soup.find_all('td'):
            element = str(element)
            all_page_data.append(element)
            
        for i in range (len(all_page_data)):
            if 'frakcióvez. h.' in all_page_data[i]:
                hierarchy[i]= 
                    
    return hierarchy
    
    
    
    
        
#Part 2: gives a data in the seeked format for us, and return 99995, if the politician has not been in committee    
    result = []
    for committee in Committee_list:
        for i in range (len(all_page_data)):

            if committee in all_page_data[i]:
                result.append(committee)
                result.append(all_page_data[i+1][4:-5])
                result.append(all_page_data[i+2][4:-5])
                result.append(all_page_data[i+3][4:-5])
    if result == []:
        result = 99995
    return result
    
    ____________
    
    
    
    
    
        values = list(row.values())
        if values [3] == 99995:
            df = df.append(row, ignore_index=True)
        else:
            start = (values[4]+'-'+values[5]+'-'+values[6])
            end   = (values[7]+'-'+values[8]+'-'+values[9])

            #Remove duplicate data
            if '20' in values[4] and '20' in values [7]:
                start = datetime.datetime.strptime(start, '%Y-%m-%d')
                end = datetime.datetime.strptime(end, '%Y-%m-%d')

                if election_2014 <= start <= election_2018:
                    print(row)
                    df = df.append(row, ignore_index=True)    